# Spark ETL with SQL Database PostgreSQL

1. Install required spark libraries
2. Create connection with PostgreSQL Database
3. Read data from PostgreSQL Database
4. Transform data
5. Write data into PostgreSQL Server

### 1- Spark Librairies

Start Spark Session and Load all the required library

In [2]:
from pyspark.sql import SparkSession

https://mvnrepository.com/artifact/org.postgresql/postgresql/42.6.0 : 'org.postgresql:postgresql:42.6.0'

In [18]:
pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [5]:
spark = SparkSession.builder \
           .appName('JDBC') \
           .config("spark.jars", "org.postgresql:postgresql:42.6.0.jar")\
           .getOrCreate()

RuntimeError: Java gateway process exited before sending its port number

In [ ]:
spark